# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.dropna
new_cities_df = cities_df.loc[(cities_df['Lat']!='NA') & (cities_df['Lat']>= -90) & (cities_df['Lat']<= 90)]
new_cities_df.head()

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,289.45,69.0,0.0,2.61,US,1.662084e+09
1,1,arraial do cabo,-22.9661,-42.0278,293.03,79.0,0.0,11.42,BR,1.662084e+09
2,2,rikitea,-23.1203,-134.9692,295.70,70.0,1.0,4.51,PF,1.662084e+09
3,3,chabahar,25.2919,60.6430,299.74,83.0,32.0,2.16,IR,1.662084e+09
4,4,lavrentiya,65.5833,-171.0000,279.66,83.0,100.0,6.40,RU,1.662084e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = new_cities_df[['Lat','Lon']]


rating = new_cities_df["Humidity"].astype(float)


In [4]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

In [5]:

heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Ideal Weather:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Humidity Greater then 30 but less than 50

# F = 1.8*(K-273) + 32.
new_cities_df['Max Temp'] = ((cities_df['Max Temp']-273) *1.8) + 32

idealWeather_df = new_cities_df.loc[
    (new_cities_df['Max Temp'] < 80) &
    (new_cities_df['Max Temp'] > 70) &
    (new_cities_df['Wind Speed'] < 10)  &
    (new_cities_df['Humidity'] < 70) 
]

idealWeather_df.head()

C:\Users\Renee holliday\AppData\Local\Temp\ipykernel_17264\4107190087.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_cities_df['Max Temp'] = ((cities_df['Max Temp']-273) *1.8) + 32


,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,vaini,-21.2000,-175.2000,70.556,65.0,1.0,4.00,TO,1.662084e+09
28,28,bethel,41.3712,-73.4140,71.024,64.0,3.0,2.39,US,1.662084e+09
51,51,east london,-33.0153,27.9116,73.832,47.0,87.0,4.44,ZA,1.662084e+09
64,64,yumen,40.2833,97.2000,70.862,26.0,52.0,8.46,CN,1.662084e+09
80,80,faanui,-16.4833,-151.7500,79.214,67.0,2.0,6.11,PF,1.662084e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = idealWeather_df

hotel_df['Hotel Name'] = ''
hotel_df.head()

C:\Users\Renee holliday\AppData\Local\Temp\ipykernel_17264\886741791.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,18,vaini,-21.2000,-175.2000,70.556,65.0,1.0,4.00,TO,1.662084e+09,
28,28,bethel,41.3712,-73.4140,71.024,64.0,3.0,2.39,US,1.662084e+09,
51,51,east london,-33.0153,27.9116,73.832,47.0,87.0,4.44,ZA,1.662084e+09,
64,64,yumen,40.2833,97.2000,70.862,26.0,52.0,8.46,CN,1.662084e+09,
80,80,faanui,-16.4833,-151.7500,79.214,67.0,2.0,6.11,PF,1.662084e+09,


In [24]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat= row['Lat']
    lon = row['Lon']
    target_coordinates = (f'{lat},{lon}')
    target_search = 'Hotel'
    target_radius = 5000
    target_type = 'hotel'

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }


    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    # print(json.dumps(places_data, indent=4, sort_keys=True))
    # break
    try:
        hotel_df.loc[index, 'Hotel Name'] = places_data['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

        

  

hotel_df.head()


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
18,18,vaini,-21.2000,-175.2000,70.556,65.0,1.0,4.00,TO,1.662084e+09,Dayspring Lodge
28,28,bethel,41.3712,-73.4140,71.024,64.0,3.0,2.39,US,1.662084e+09,Hampton Inn Danbury
51,51,east london,-33.0153,27.9116,73.832,47.0,87.0,4.44,ZA,1.662084e+09,Blue Lagoon Hotel
64,64,yumen,40.2833,97.2000,70.862,26.0,52.0,8.46,CN,1.662084e+09,
80,80,faanui,-16.4833,-151.7500,79.214,67.0,2.0,6.11,PF,1.662084e+09,Four Seasons Resort Bora Bora


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [33]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
# Add the layer to the map
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))